In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

In [15]:
%pip install aif360

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python38\python.exe -m pip install --upgrade pip' command.


In [16]:
%pip install cvxpy==0.4.11

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python38\python.exe -m pip install --upgrade pip' command.


In [21]:
url = "https://raw.githubusercontent.com/cchanyu/MachineLearning/main/SheetData/average-height-of-men-for-selected-countries.csv"
df = pd.read_csv(url, sep=',')

In [18]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

In [22]:
df.head()

,Entity,Code,Year,Human Height (University of Tuebingen (2015))
0,Afghanistan,AFG,1870,168.4
1,Afghanistan,AFG,1880,165.7
2,Afghanistan,AFG,1930,166.8
3,Albania,ALB,1880,170.1
4,Albania,ALB,1890,169.8


In [53]:
from aif360.datasets import StandardDataset
class ModifiedDataset(StandardDataset):

    def __init__(self, label_name='Human Height (University of Tuebingen (2015))', favorable_classes=[1],\
        protected_attribute_names=['Human Height (University of Tuebingen (2015))'],\
        privileged_classes=[[0],[0],[0],[0]],\
        instance_weights_name=None,\
        categorical_features=[],\
        features_to_keep=[], features_to_drop=[],\
        na_values=[], custom_preprocessing=None,\
        metadata=None):

        super(ModifiedDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

In [54]:
dataset_orig = ModifiedDataset(protected_attribute_names=['Human Height (University of Tuebingen (2015))'],
privileged_classes=[lambda x: x >= 170],      
features_to_drop=['Entity', 'Code']) 

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'Human Height (University of Tuebingen (2015))': 1}]
unprivileged_groups = [{'Human Height (University of Tuebingen (2015))': 0}]

In [55]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = -1.000000


In [56]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

divide by zero encountered in double_scalars
divide by zero encountered in double_scalars


In [57]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = -1.000000
